In [1]:
!pip install wandb
import wandb
!pip install pytorch_lightning
import pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import torchvision.models as models
import torch
import pandas as pd

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:

####Update variables 
dataset = '2a' #choose dataset to predict on; 1a 1b 2a or 2b

model_to_use = 'final' #select which model: final or finetune

base_dir = '/content/gdrive/MyDrive/' #update file path for HurricaneDamage Folder

own_data_dir = None #update if you want to predict on your own data e.g. your own subset of one of the datasetse. Needs to be in folder format Train/Damaged_Y, Train/Damaged_N, Test/Damaged_Y, Test/Damaged_N

output_location= None #update with path if you want to save predictions


In [5]:
import sys
sys.path.insert(0, base_dir + '/HurricanDamage_Data')

from src import NOADataModule  , ImagePredictionLogger, ImagePredictionLogger_table,  TransferLearning

In [6]:
#Parameters set for our Final Model config; using ResNet50 architecture
bs = 160
LR = 0.006607
weights = [0.2,1]
optimiser = 'Adam'

model = TransferLearning(model=models.resnet50(pretrained=True), learning_rate = LR, optimiser = optimiser, weights = weights , av_type = 'micro' )

/usr/local/lib/python3.7/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


In [8]:
#Loads the model 
if model_to_use == 'final':
  PATH = base_dir + 'HurricanDamage_Data/models/Final_model' 
elif model_to_use =='finetune':
  PATH = base_dir + 'HurricanDamage_Data/models/Final_PlusDelta_finetune'  
else:
  print('Error: Which model do you want to use?')
  sys.exit()

model.load_state_dict(torch.load(PATH))
model.eval()

TransferLearning(
  (accuracy): Accuracy()
  (loss): CrossEntropyLoss()
  (recall): Recall()
  (prec): Precision()
  (jacq_ind): JaccardIndex()
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, mom

In [9]:
#Set up data dirs
if own_data_dir is None:
  dir = '/content/gdrive/MyDrive/HurricanDamage_Data/dataset_' + dataset
else:
  dir = own_data_dir

#Set up data module
dm = NOADataModule(data_dir = dir , H= 224, W = 224, data_mean = (0.416,0.416 , 0.36 ), data_sd = (0.215 ,0.21 , 0.21 ), batch_size=bs , test_only= 'No')
dm.prepare_data()
dm.setup()

Train size: 1
Valid size: 1
Test size: 3108


In [ ]:
#Extract the Ground truth and Predictions 
ground_truth = [] 
list_output = []
predictions = []

test_dataset = dm.test_dataset
for sample in test_dataset:
  label = sample[1]
  data = sample[0]
  ground_truth.append(label)
print('Ground truth list created')

test_ids = dm.test_ids
print('Num of Test IDs is {}, Num of Ground Truth labels is {}'.format(len(test_ids), len(ground_truth)) )


model.eval()
for x in dm.test_dataloader():
   output = model(x[0])
   x = torch.argmax(output, dim=1)
   x= torch.flatten(x)
   list_output.append(x)

for x in list_output:
  x= x.tolist()
  # print(x)
  for y in x:
    predictions.append(y)
print('Prediction list created')

In [ ]:
#create df of predictions and save 
df = pd.DataFrame( {'TestId': test_ids , 'GroundTruth' :ground_truth, 'Preds' : predictions   })
if output_location is not None: 
  df.to_csv(output_location )

In [ ]:

df.head()